In [69]:
import pandas as pd

In [70]:
import os
import argparse 
from datetime import datetime
from unittest import result
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import preprocessing
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import scanpy as sc


config_kwargs = {
    'comb_num' : 1,
    'save_dir' : '/data/nicole/PRnet/FDA_3044/drug/',
    'results_dir' : '/data/nicole/PRnet/FDA_3044/drug/',
    'split_key' : 'FDA961_split',
    'x_dimension' : 978,
    'n_genes' : 20,
    'obs_key' : 'cov_drug_dose_split',
    'cell_line': 'CRC'
}  

def parse_args():
    parse = argparse.ArgumentParser(description='Single Cell Drug combinations')  
    parse.add_argument('--split_key', default='CRC_split1', type=str, help='split key of data')  
    args = parse.parse_args()  
    return args


def pearson_mean(data1, data2):
    sum_pearson_1 = 0
    sum_pearson_2 = 0
    for i in range(data1.shape[0]):
        pearsonr_ = pearsonr(data1[i], data2[i])
        sum_pearson_1 += pearsonr_[0]
        sum_pearson_2 += pearsonr_[1]
    return sum_pearson_1/data1.shape[0], sum_pearson_2/data1.shape[0]

def pearson_list(data1, data2):
    pearson_list = np.zeros(data1.shape[0])
    for i in range(data1.shape[0]):
        pearsonr_ = pearsonr(data1[i], data2[i])
        pearson_list[i] = pearsonr_[0]
    return pearson_list

def spearman_list(data1, data2):
    pearson_list = np.zeros(data1.shape[0])
    for i in range(data1.shape[0]):
        pearsonr_ = spearmanr(data1[i], data2[i])
        pearson_list[i] = pearsonr_[0]
    return pearson_list


def r2_mean(data1, data2):
    sum_r2_1 = 0
    for i in range(data1.shape[0]):
        r2_score_ = r2_score(data1[i], data2[i])
        sum_r2_1 += r2_score_           
    return sum_r2_1/data1.shape[0]

def mse_mean(data1, data2):
    sum_mse_1 = 0
    for i in range(data1.shape[0]):
        mse_score_ = mean_squared_error(data1[i], data2[i])
        sum_mse_1 += mse_score_           
    return sum_mse_1/data1.shape[0]

def z_score(control_array, drug_array):
    scaler = preprocessing.StandardScaler()
    array_all = np.concatenate((control_array, drug_array),axis=0)
    scaler.fit(array_all)
    control_array_z = scaler.transform(control_array)
    drug_array_z = scaler.transform(drug_array)
    return control_array_z, drug_array_z

def contribution_df(data):
    data['cov_drug_dose_name'] = data.index
    data['cell_type'] = data.cov_drug_dose_name.apply(lambda x: str(x).split('_')[0])
    data['drug'] = data.cov_drug_dose_name.apply(lambda x: '_'.join(str(x).split('_')[1:-1]))
    data['dose'] = data.cov_drug_dose_name.apply(lambda x: str(x).split('_')[-1])
    data['cov_drug_name'] = data.cov_drug_dose_name.apply(lambda x: '_'.join(str(x).split('_')[:-1]))
    return data

# This file consists of useful functions that are related to cmap
def computecs(qup, qdown, expression):
    '''
    This function takes qup & qdown, which are lists of gene
    names, and  expression, a panda data frame of the expressions
    of genes as input, and output the connectivity score vector
    '''
    r1 = ranklist(expression)
    if qup and qdown:
        esup = computees(qup, r1)
        esdown = computees(qdown, r1)
        w = []
        for i in range(len(esup)):
            if esup[i]*esdown[i] <= 0:
                w.append(esup[i]-esdown[i])
            else:
                w.append(0)
        return pd.DataFrame(w, expression.columns)
    elif qup and qdown==None:
        esup = computees(qup, r1)
        return pd.DataFrame(esup, expression.columns)
    elif qup == None and qdown:
        esdown = computees(qdown, r1)
        return pd.DataFrame(esdown, expression.columns)
    else:
        return None

def computees(q, r1):
    '''
    This function takes q, a list of gene names, and r1, a panda data
    frame as the input, and output the enrichment score vector
    '''
    if len(q) == 0:
        ks = 0
    elif len(q) == 1:
        ks = r1.loc[q,:]
        ks.index = [0]
        ks = ks.T
#print(ks)
    else:
        n = r1.shape[0]
        sub = r1.loc[q,:]
        J = sub.rank()
        a_vect = J/len(q)-sub/n
        b_vect = (sub-1)/n-(J-1)/len(q)
        a = a_vect.max()
        b = b_vect.max()
        ks = []
        for i in range(len(a)):
            if a[i] > b[i]:
                ks.append(a[i])
            else:
                ks.append(-b[i])
#print(ks)
    return ks
def ranklist(DT):
    # This function takes a panda data frame of gene names and expressions
    # as an input, and output a data frame of gene names and ranks
    ranks = DT.rank(ascending=False, method="first")
    return ranks

def condition_fc_groups_by_cov(
    data,
    groupby,
    control_group,
    covariate
):

    condition_exp_mean = {}
    control_exp_mean = {}
    fold_change = {}

    cov_categories = data[covariate].unique()
    for cov_cat in cov_categories:
        #name of the control group in the groupby obs column
        control_group_cov = '_'.join([cov_cat, control_group])

        #subset adata to cells belonging to a covariate category
        adata_cov_df = data[data[covariate]==cov_cat]
        adata_cov_df['condition'] = data[groupby]


        control_mean = adata_cov_df[adata_cov_df.cov_drug_name == control_group_cov].mean(numeric_only=True)
        control_exp_mean[control_group_cov] = control_mean

        for cond, df in tqdm(adata_cov_df.groupby('condition')): 
            if df.shape[0] != 0 :
                if cond != control_group_cov:
                    drug_mean = df.mean(numeric_only=True)
                    fold_change[cond] = drug_mean-control_mean
                    condition_exp_mean[cond] = drug_mean

    return condition_exp_mean, control_exp_mean, fold_change




'''
pre_array = np.genfromtxt(config_kwargs['save_dir']+config_kwargs['split_key']+'y_pre_array.csv', delimiter=',')
f = open(config_kwargs['save_dir']+config_kwargs['split_key']+'cov_drug_array.csv',"r")
lines = f.readlines()
cov_drug = [x.strip() for x in lines]
'''
print('Loading data...')

drug_z_mean_df=pd.read_csv(config_kwargs['save_dir']+'FDA_'+'drug'+'_fold-change_2W.csv', index_col=0)
drug_fc_mean_df=pd.read_csv(config_kwargs['save_dir']+'FDA_'+'drug'+'_z-score_2W.csv', index_col=0)

Loading data...


In [71]:
gene_list_df = pd.read_excel('./datasets/SCLC/HBEXP001299.xlsx')

In [72]:
up_gene = gene_list_df[gene_list_df['Log2 (FC_avg)']>0]['Gene name'].to_list()
down_gene = gene_list_df[gene_list_df['Log2 (FC_avg)']<0]['Gene name'].to_list()

In [17]:
up_gene_df = pd.read_csv('./datasets/disease DEG/NASH_up_short.csv')
down_gene_df = pd.read_csv('./datasets/disease DEG/NASH_down_short.csv')

In [23]:
up_gene_df = pd.read_csv('./datasets/disease DEG/NASH_up.csv')
down_gene_df = pd.read_csv('./datasets/disease DEG/NASH_down.csv')

In [29]:
up_gene_df = pd.read_csv('./datasets/disease DEG/FIBROSIS_up.csv')
down_gene_df = pd.read_csv('./datasets/disease DEG/FIBROSIS_down.csv')

In [46]:
up_gene_df = pd.read_csv('./datasets/disease DEG/HUA_up.csv')
down_gene_df = pd.read_csv('./datasets/disease DEG/HUA_down.csv')

In [53]:
up_gene_df = pd.read_csv('./datasets/disease DEG/BROWNING_up.csv')
down_gene_df = pd.read_csv('./datasets/disease DEG/BROWNING_down.csv')

In [59]:
up_gene_df = pd.read_csv('./datasets/disease DEG/BROWNING_up_short.csv')
down_gene_df = pd.read_csv('./datasets/disease DEG/BROWNING_down_short.csv')

In [66]:
gene_list_df = pd.read_csv('./datasets/SCLC/scc1.csv')
gene_list_df =  gene_list_df[~gene_list_df.Gene.isna()]
up_gene = gene_list_df[gene_list_df.log2FoldChange>0].Gene.to_list()
down_gene = gene_list_df[gene_list_df.log2FoldChange<0].Gene.to_list()

In [60]:
up_gene = up_gene_df.name.to_list()
down_gene = down_gene_df.name.to_list()

In [73]:
gene_list = drug_z_mean_df.columns.to_list()
up_gene_list = list(set(gene_list) & set(up_gene))
down_gene_list = list(set(gene_list) & set(down_gene))

In [74]:
z_es_array = computecs(up_gene_list,down_gene_list,drug_z_mean_df.T)[0].values
fc_es_array = computecs(up_gene_list,down_gene_list,drug_fc_mean_df.T)[0].values


In [75]:
es_df = pd.DataFrame(index=drug_z_mean_df.index,columns=['z-score es', 'fold-change es'])
es_df['z-score es']=z_es_array
es_df['fold-change es']=fc_es_array
es_df['fold-change+z-score es'] = es_df['z-score es']+es_df['fold-change es']

In [22]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)     #NASH_short

,z-score es,fold-change es,fold-change+z-score es
Ethoxzolamide,0.742651,0.680069,1.422720
(+)-Camphor,0.536080,0.801602,1.337683
atorvastatin,0.539762,0.783873,1.323635
"2,6-Dihydroxypurine",0.542130,0.734165,1.276295
Alibendol,0.509709,0.752976,1.262685
Ansamitocin,0.580401,0.671064,1.251465
Squalane,0.524374,0.723295,1.247669
Pravastatin,0.541449,0.699334,1.240783
Atorvastatin,0.599317,0.637879,1.237196
Levothyroxine,0.646171,0.579923,1.226094


In [28]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)     #NASH_all Repaglinide Teneligliptin	

,z-score es,fold-change es,fold-change+z-score es
Topiramate,0.335431,0.615569,0.950999
Orcinol,0.378165,0.483187,0.861351
Maltotriose,0.390055,0.394325,0.784380
Ala-Gln,0.335914,0.430723,0.766637
Cyclandelate,0.267637,0.486622,0.754258
D-Pantethine,0.325227,0.401992,0.727219
Repaglinide,0.279078,0.444550,0.723628
Trans-Tranilast,0.328250,0.384750,0.713000
Valbenazine,0.304676,0.403804,0.708480
4-Aminobutyric,0.230862,0.476929,0.707792


In [39]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)     #FIBROSIS  Silymarin

,z-score es,fold-change es,fold-change+z-score es
Quercitrin,0.292469,0.470368,0.762837
Delamanid,0.359442,0.402857,0.762299
Equol,0.343214,0.401774,0.744988
Ganirelix,0.300970,0.423080,0.724050
DL-Penicillamine,0.292731,0.418457,0.711188
8-Hydroxyquinoline,0.267753,0.423368,0.691121
Galanthamine,0.309251,0.380136,0.689387
Silymarin,0.326855,0.361804,0.688659
Pimobendan,0.297988,0.389166,0.687153
Macitentan,0.295651,0.390180,0.685831


In [51]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)     #HUA

,z-score es,fold-change es,fold-change+z-score es
Benserazide,0.241147,0.392710,0.633858
Amlexanox,0.000000,0.453856,0.453856
Butenafine,0.000000,0.447447,0.447447
Reboxetine,0.000000,0.413689,0.413689
Simeprevir,0.000000,0.413484,0.413484
Esomeprazole,0.000000,0.404730,0.404730
Cefmetazole,0.000000,0.404298,0.404298
Bromocriptine,0.000000,0.401589,0.401589
Aclidinium,0.000000,0.397677,0.397677
Upadacitinib,0.000000,0.391998,0.391998


In [58]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)     #BROWNING

,z-score es,fold-change es,fold-change+z-score es
Fexofenadine,0.284151,0.363765,0.647916
Triclosan,0.270846,0.374301,0.645147
Carteolol,0.263631,0.355587,0.619219
Latamoxef,0.326090,0.292198,0.618288
Anethole,0.248310,0.367722,0.616032
Rifampin,0.237204,0.357997,0.595201
Deserpidine,0.235963,0.358246,0.594209
Lumiracoxib,0.262131,0.322113,0.584245
Sulfabenzamide,0.222707,0.347957,0.570665
Secnidazole,0.284336,0.261791,0.546127


In [64]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)     #BROWNING_short

,z-score es,fold-change es,fold-change+z-score es
Ceftaroline,0.582735,0.817715,1.400450
α-Hederin,0.473759,0.861243,1.335003
Midecamycin,0.441754,0.881279,1.323033
Naringin,0.440315,0.851272,1.291587
Dioscin,0.434449,0.852719,1.287168
Venlafaxine,0.405067,0.881957,1.287025
Rifaximin,0.367336,0.912295,1.279631
Lactose,0.418668,0.842242,1.260910
Amphotericin,0.471432,0.767829,1.239261
Sanguinarine,0.411638,0.772844,1.184482


In [76]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)  #SCLC_all

,z-score es,fold-change es,fold-change+z-score es
Indometacin,0.349518,0.394042,0.743560
Tiagabine,0.377312,0.349829,0.727141
Bornyl,0.350951,0.373660,0.724611
Phenolphthalein,0.402039,0.310240,0.712278
Indoximod,0.361997,0.332344,0.694341
Clonixin,0.339785,0.331482,0.671268
L-SelenoMethionine,0.319493,0.349798,0.669291
β-Alanine,0.321195,0.326835,0.648030
Minoxidil,0.363738,0.282742,0.646480
Morin,0.325115,0.320548,0.645662


In [80]:
drug_info = pd.read_excel('./datasets/FDA_961/FDA-approved-Drug-Library-96-well.xlsx',index_col=0)

In [81]:
drug_info['drug'] = drug_info.Name.apply(lambda x: str(x).split(' ')[0])

In [84]:
drug_info[drug_info.drug=='Phenolphthalein']

,Name,Plate Location,Rack Number,Formulation,Status,Pharmacopoeia,FDA NDA/ANDA,Indication,Target,Pathway,...,URL,Formula,Form,Synonyms,SMILES,ALogP,HBA_Count,HBD_Count,RotatableBond,drug
Cat,,,,,,,,,,,,,,,,,,,,,
S5395,Phenolphthalein,b4,L1300-24,in 10mM DMSO,"DMF,CFDA,NDC","Martindale:The Extra Pharmacopoeia,USP42-NF37,...",NaN,Gastroenterology,Fluorescent Dyes/Probes,Others,...,http://selleckchem.com/products/.html,C20H14O4,free base,NaN,OC1=CC=C(C=C1)C2(OC(=O)C3=C2C=CC=C3)C4=CC=C(O)...,4.006,2.0,2.0,2.0,Phenolphthalein


In [40]:
import pandas as pd
gene_list_df = pd.read_csv('./datasets/SCLC/scc1.csv')
gene_list_df =  gene_list_df[~gene_list_df.Gene.isna()]

In [41]:
up_gene = gene_list_df[gene_list_df.log2FoldChange>0].Gene.to_list()
down_gene = gene_list_df[gene_list_df.log2FoldChange<0].Gene.to_list()

In [42]:
gene_list = drug_z_mean_df.columns.to_list()
up_gene_list = list(set(gene_list) & set(up_gene))
down_gene_list = list(set(gene_list) & set(down_gene))

In [43]:
z_es_array = computecs(down_gene_list,up_gene_list,drug_z_mean_df.T)[0].values
fc_es_array = computecs(down_gene_list,up_gene_list,drug_fc_mean_df.T)[0].values

In [44]:
es_df = pd.DataFrame(index=drug_z_mean_df.index,columns=['z-score es', 'fold-change es'])
es_df['z-score es']=z_es_array
es_df['fold-change es']=fc_es_array
es_df['fold-change+z-score es'] = es_df['z-score es']+es_df['fold-change es']

In [45]:
es_df.sort_values(by='fold-change+z-score es', ascending=False).head(20)   #SCC  Palbociclib	Vismodegib Trametinib

,z-score es,fold-change es,fold-change+z-score es
Letermovir(AIC246),0.191152,0.279263,0.470415
Etonogestrel,0.181010,0.263014,0.444024
Xipamide,0.180537,0.257239,0.437777
Creatinine,0.167849,0.229804,0.397652
Rifampin,0.170989,0.226071,0.397061
Gossypol,0.176356,0.220395,0.396751
Amiloride,0.157180,0.224838,0.382019
Eliglustat,0.153825,0.228163,0.381988
Palbociclib,0.154532,0.221073,0.375605
Levetiracetam,0.156049,0.218954,0.375004
